# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [4]:
import pandas as pd
import numpy as np
from scipy import stats

In [6]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


#1.What test is appropriate for this problem? Does CLT apply?

Let's take a look at the data and how they are divided. 

In [11]:
print('Total number of records in this sample is {}'.format(len(data)))
print('Total number of black race in this sample is {}'.format(len(data[data['race']=='b'])))
print('Total number of white race in this sample is {}.'.format(len(data[data['race']=='w'])))

Total number of records in this sample is 4870
Total number of black race in this sample is 2435
Total number of white race in this sample is 2435.


As we see, the sample size is pretty huge and there is a finite level of variance in the sample size. Hence we can say that the Central Limit Theorem does apply here. Also the mean of all samples will be approximately equal to the population mean. As we are testing with two different sample (black and white race), the two-sample t-test would be more appropriate.

#2. What are the null and alternate hypotheses?

Null hypothesis (H0)
The null hypothesis states that a population parameter (such as the mean, the standard deviation, and so on) is equal to a hypothesized value. The null hypothesis is often an initial claim that is based on previous analyses or specialized knowledge.


Alternative Hypothesis (H1)
The alternative hypothesis states that a population parameter is smaller, greater, or different than the hypothesized value in the null hypothesis. The alternative hypothesis is what you might believe to be true or hope to prove true.

#3. Compute margin of error, confidence interval, and p-value.

In [16]:
data_racew = data[data.race == 'w']
data_raceb = data[data.race == 'b']
n_w = len(data_racew)
p_w = sum(data_racew.call) / n_w
n_b = len(data_raceb)
p_b = sum(data_raceb.call) / n_b

print('The sample mean for \'b\' race callbacks is {:.2f}.'.format(data_raceb['call'].mean()))
print('The sample mean for \'w\' race callbacks is {:.2f}.'.format(data_racew['call'].mean()))
print('The sample mean difference is {:.3f}'.format(data_racew['call'].mean() - data_raceb['call'].mean()))
print('The difference in proportion is ', p_w - p_b)

The sample mean for 'b' race callbacks is 0.06.
The sample mean for 'w' race callbacks is 0.10.
The sample mean difference is 0.032
The difference in proportion is  0.0320328542094


Let's calculate the p of comined sum
$\bar{p} = \frac{n_w p_w + n_b p_b}{n_w+n_b}$

Then we will calculate the sigma of differences
$\sigma_{p_w-p_b} = \sqrt{\frac{\bar{p}(1-\bar{p})}{n_w} + \frac{\bar{p}(1-\bar{p})}{n_b}}$

In [17]:
p_all = (sum(data_racew.call) + sum(data_raceb.call)) / (n_w + n_b)
sigma = ((p_all*(1-p_all) / n_w) + (p_all*(1-p_all) / n_b)) ** 0.5
confidence = 0.95
confidence_z = 1 - (1-confidence)/2
ME = sigma * stats.norm.ppf(confidence_z)
CI = (-ME, ME)
print('The 95% confidence interval for the difference in proportion is ', CI)
print('Margin of error is ', (ME - (-ME))/2)

The 95% confidence interval for the difference in proportion is  (-0.015281631502169232, 0.015281631502169232)
Margin of error is  0.0152816315022


In [18]:
mean_racew = data[data.race == 'w'].call.mean()
var_racew = data[data.race == 'w'].call.var()
mean_raceb = data[data.race == 'b'].call.mean()
var_raceb = data[data.race == 'b'].call.var()

SE = np.sqrt(var_racew/n_w + var_raceb/n_b)

print('Mean of white-sounding names callback is {:.2f}'.format(mean_racew))
print('Mean of black-sounding names callback is {:.2f}'.format(mean_raceb))
print('Standard error is: ', SE)

two_sample = stats.ttest_ind(data_raceb['call'], data_racew['call'])
print('The t statistic is %.3f and the p-value is %.6f.' % two_sample)

Mean of white-sounding names callback is 0.10
Mean of black-sounding names callback is 0.06
Standard error is:  0.00778490682688
The t statistic is -4.115 and the p-value is 0.000039.


#4. Write a story describing the statistical significance in the context or the original problem.

There is a significant difference between the number of callback received by black sounding name compared to white sounding names based on the p-value and 95% confidence interval. Also the callback received by white sounding names are .04 more than people with black sounding names. Even though the result is not practically significant in the context or the original problem, but it seems the true difference in calls might not be because of the race factor alone.

#5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
Definetely Not. Race and name might play as one of the factor for receiving the callback but not the only factor. There might be some other features such as the education, experience and how each individual to qualified enough to do that particular job. To complete the analysis we would need to consider all the above factors.
